# Zonal Statistics with ASTER and MODIS Thermal Infrared Radiance Imagery

In [1]:
# xarray family of packages for working with raster data
import xarray as xr
import xrspatial as xrs
import rioxarray

# aster and modis functions for unit conversion of digital number to radiance and brightness temperature
import aster_utils 
import modis_utils

# our favorite python packages for arrays, data frames, and plotting
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

/home/spestana/opt/anaconda3/envs/goes-linux/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/spestana/opt/anaconda3/envs/goes-linux/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/spestana/opt/anaconda3/envs/goes-linux/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/spestana/opt/anaconda3/envs/goes-linux/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


---
**Open and clean-up input datasets**

Use rioxarray to open a GeoTIFF of MODIS Radiance

In [2]:
modis_rad_filepath = '/storage/MODIS/Tuolumne_MOD021KM/MOD021KM.A2017111.1850.061.2017314062948.tif'
modis_ds = xr.open_rasterio(modis_rad_filepath)

Open the coincident ASTER thermal infrared radiance GeoTIFF image we want to compare with

In [3]:
aster_rad_filepath = '/storage/spestana/ASTER/AST_L1T/geotiff/T/T_band14_Tuolumne-and-CUES/AST_L1T_00304212017185107_20170422082541_26960_ImageData14.tif'
aster_src = xr.open_rasterio(aster_rad_filepath)

Define a working area her in local UTM coordinates to clip all three rasters to the same common area

In [4]:
utm_northings_max = 4205000
utm_northings_min = 4185000
utm_eastings_max = 305000
utm_eastings_min = 285000

geometries = [
    {
        'type': 'Polygon',
        'coordinates': [[
            [utm_eastings_min, utm_northings_max],
            [utm_eastings_max, utm_northings_max],
            [utm_eastings_max, utm_northings_min],
            [utm_eastings_min, utm_northings_min]
        ]]
    }
]

Clean up the ASTER image by replacing nodata values with NaN, removing an extra dimension, converting the digital number values to radiance values, and finally clipping to the geometry defined above.

In [5]:
# Replace the nodatavals with NaN, squeeze out the band dim we don't need
aster_src = aster_src.where(aster_src!=aster_src.nodatavals, np.nan).squeeze()
# Convert ASTER DN to Radiance
aster_band = 14
aster_rad = aster_utils.tir_dn2rad(aster_src, aster_band)
# set crs back
aster_rad.rio.set_crs(aster_src.crs, inplace=True)
# clip to geometry
aster_rad_clipped = aster_rad.rio.clip(geometries).rename('aster')

Use reproject_match to align the MODIS radiance & zones raster to the ASTER image, clip to the geometry defined above.

In [ ]:
# convert from DN to Radiance and Brightness Temperature
modis_rad_tb = modis_utils.emissive_convert_dn(modis_ds)
# Select a single band from the MODIS dataset
# make sure to use MODIS band 31 (here index 10, it is around 11 microns)
# See a list of band numbers with: modis_ds_repr_match.band_names.split(',')
modis_band_index = 10
modis_rad_tb_single_band = modis_rad_tb.isel(band=modis_band_index)
# Create "zone_labels" by numbering each MODIS pixel
n_rows, n_cols = modis_rad_tb_single_band.radiance.shape
modis_rad_tb_single_band['modis_zones'] = (('y', 'x'), np.reshape(np.arange(n_rows*n_cols), (n_rows, n_cols)))
# Change datatype to float, this is requried for reproject match apparently (?) change back to int later
modis_rad_tb_single_band['modis_zones'] = modis_rad_tb_single_band.modis_zones.astype('float32')
# Add CRS info to this new "modis_zones" data array by copying over attributes from another data array
zones_attrs = modis_rad_tb_single_band.radiance.attrs.copy()
# edit long name attribute for modis zones data array
zones_attrs['long_name'] = 'modis_zone_labels'
# Add the attributes
modis_rad_tb_single_band.modis_zones.attrs = zones_attrs
## use Reproject_Match to reproject the GOES geotiff into the same CRS as the ASTER geotiff
modis_rad_tb_single_band_repr_match = modis_rad_tb_single_band.rio.reproject_match(aster_src)
# switch modis_zones back to int64
modis_rad_tb_single_band_repr_match['modis_zones'] = modis_rad_tb_single_band_repr_match.modis_zones.astype('int64')
# clip out anything that is nan in the ASTER image
modis_rad_tb_single_band_repr_match = modis_rad_tb_single_band_repr_match.where(np.isnan(aster_src.values) == False)
## set crs
#modis_ds_repr_match.rio.set_crs(aster_src.crs, inplace=True)
# clip to geometry
modis_rad_tb_single_band_repr_match = modis_rad_tb_single_band_repr_match.rio.clip(geometries)
# remove nodata value
modis_repr = modis_rad_tb_single_band_repr_match.where(modis_rad_tb_single_band_repr_match.tb_c != modis_rad_tb_single_band_repr_match.tb_c._FillValue)
#########   

Plot each of these images:

In [ ]:
fig, [ax1, ax2, ax3] = plt.subplots(nrows=1,ncols=3, figsize=(15,4), tight_layout=True)

aster_rad_clipped.plot(ax=ax1, cmap='Greys', vmin=5, vmax=8, cbar_kwargs={'label': 'Radiance\n[$W m^{-2} sr^{-1} um^{-1}$]'})
ax1.set_title('ASTER\nBand 14 Radiance')

modis_repr.radiance.plot(ax=ax2, cmap='Greys', vmin=5, vmax=8, cbar_kwargs={'label': 'Radiance\n[$W m^{-2} sr^{-1} um^{-1}$]'})
ax2.set_title('MODIS \nBand 31 Radiance')

modis_repr.modis_zones.plot(ax=ax3, cbar_kwargs={'label': 'Zone #'})
ax3.set_title('MODIS\nPixel Footprint "zones"')

for this_ax in [ax1, ax2, ax3]:
    this_ax.set_ylabel('Northings (m, UTM 11N)')
    this_ax.set_xlabel('Eastings (m, UTM 11N)')

---
**Compute zonal statistics and format results**

Compute zonal statistics from the ASTER image, using the GOES zone labels:

In [ ]:
zonalstats = xrs.zonal.stats(goes_zones_repr, 
                             aster_rad_clipped, 
                             stat_funcs=['mean', 'max', 'min', 'std', 'var', 'count'])

# Preview the resulting dataframe
zonalstats.head()

In [ ]:
# Convert zonal statistics dataframe to xarray dataset
zonalstats = xr.Dataset(zonalstats)

# Preview the dataset
zonalstats

Define a function to map the results from xrs.zonal.stats() back into the original zones grid

In [ ]:
def mapZonalStats(zones, zonalstats, stat_name):
    ''' Function for mapping the zonal statistics back to the original grid to get a 2D map of the chosen statistic'''
    # create an empty array for this summary stat
    zonal_stat = np.zeros_like(zones.values, dtype=np.float64)

    # for each zone
    for zone_n in zonalstats.dim_0.values:
        # get the summary stat for that zone, 
        # and assign it to the correct locations in the zonal_stat array
        try:
            zonal_stat[zones.values==zone_n] = zonalstats['{}'.format(stat_name)].sel(dim_0=zone_n).values
        except: #MaskError: Cannot convert masked element to a Python int.
            zonal_stat[zones.values==zone_n] = -9999

    # convert this to an xarray data array with the proper name
    zonal_stat_da = xr.DataArray(zonal_stat, 
                                 dims=["y", "x"],
                                 coords=dict(
                                             x=(["x"], zones.x),
                                             y=(["y"], zones.y),
                                             ),
                                 name='zonal_{}'.format(stat_name))
    # remove nodata values
    zonal_stat_da = zonal_stat_da.where(zonal_stat_da!=-9999, np.nan)

    return zonal_stat_da

In [ ]:
# Map each zonal stat back into the original zones grid
zonal_means = mapZonalStats(goes_zones_repr, zonalstats, 'mean')
zonal_max = mapZonalStats(goes_zones_repr, zonalstats, 'max')
zonal_min = mapZonalStats(goes_zones_repr, zonalstats, 'min')
zonal_std = mapZonalStats(goes_zones_repr, zonalstats, 'std')
zonal_var = mapZonalStats(goes_zones_repr, zonalstats, 'var')
zonal_count = mapZonalStats(goes_zones_repr, zonalstats, 'count')

Compute the difference between GOES Radiance and the ASTER zonal mean radiance

In [ ]:
# Compute the difference between GOES Radiance and the ASTER zonal mean radiance
mean_diff = goes_rad_repr.values - zonal_means.values
# Create a data array for the mean difference values
mean_diff_da = xr.DataArray(mean_diff, name='mean_diff', dims=["y", "x"])

Merge all zonal stats back with the original ASTER data to create a single dataset

In [ ]:
# Merge all the zonal statistics data arrays and the mean difference data array
ds = xr.merge([aster_rad_clipped, goes_rad_repr, goes_zones_repr, zonal_means, zonal_max, zonal_min, zonal_std, zonal_var, zonal_count, mean_diff_da])
# Clip this dataset to the ASTER image extent
ds = ds.where(~np.isnan(aster_rad_clipped))
# Remove data where we have overlap between a GOES pixel footprint "zone" and the edge of the ASTER image
# Use the "zonal_count" (the number of ASTER pixels in each GOES pixel footprint) to determine this
# Example: 800 90m ASTER pixels is 800x90x90 square meters, or approximately 6.48 square kilometers or,
# about 2.5x2.5 km, about the size of a full GOES-16 ABI pixel here
# NOTE: The number of ASTER pixels per GOES pixel footprint is not constant, so there are some "edge pixels" still included here
ds = ds.where(ds.zonal_count > 800)

Plot all the results:

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols=4, figsize=(15,9), tight_layout=True)
ax = ax.flatten()

for i, data_var in enumerate(ds):
    ds[str(data_var)].plot(ax=ax[i])
    ax[i].set_title(str(data_var))

---

**Wrap the above into a function**

In [ ]:
def upscale_aster_goes_rad_zonal_stats(aster_rad_filepath, goes_rad_filepath, goes_zones_filepath, bounding_geometry, zonal_count_threshold=None, output_filepath=None):
    '''Given an ASTER thermal infrared radiance GeoTiff image, a GOES ABI thermal infrared radiance GeoTiff image,
       and a GOES ABI GeoTiff image defining pixel footprint "zones", compute zonal statistics for each GOES pixel
       footprint. Compute the difference between the GOES ABI radiance and ASTER zonal mean radiance.
       Return a dataset, or save dataset to a netcdf file, with the three input dataarrays plus zonal statistic data arrays.'''
    
    ### Open and clean-up input datasets ###
    # Use rioxarray to open a GeoTIFF of GOES-16 ABI Radiance that has been orthorectified for our study area
    goes_rad = xr.open_rasterio(goes_rad_filepath)
    # Open its associated GOES "zone_labels" GeoTIFF raster that was generated at the same time the image was orthorectified
    goes_zones = xr.open_rasterio(goes_zones_filepath)
    # Open the coincident ASTER thermal infrared radiance GeoTIFF image we want to compare with
    aster_src = xr.open_rasterio(aster_rad_filepath)
    
    ### Use reproject_match to align the GOES radiance and GOES zones rasters to the ASTER image, clip to the geometry defined above, and set zone values to integers. ###
    # Reproject match goes rad raster to aster, clip to geometry,  and squeeze out extra dim
    goes_rad_repr = goes_rad.rio.reproject_match(aster_src).rio.clip(bounding_geometry).squeeze().rename('goes')
    # convert GOES radiance (mW m^-2 sr^-1 1/cm^-1) to match MODIS and ASTER (W m^-2 sr^-1 um^-1)
    goes_rad_repr = (goes_rad_repr / 1000) * (61.5342/0.7711)
    # Reproject match goes zones raster to aster, clip to geometry, set datatype to integer, and squeeze out extra dim
    goes_zones_repr = goes_zones.rio.reproject_match(aster_src).rio.clip(bounding_geometry).astype('int').squeeze().rename('zones')
    
    ### Clean up the ASTER image by replacing nodata values with NaN, removing an extra dimension, converting the digital number values to radiance values, and finally clipping to the geometry defined above. ###
    # Replace the nodatavals with NaN, squeeze out the band dim we don't need
    aster_src = aster_src.where(aster_src!=aster_src.nodatavals, np.nan).squeeze()
    # Convert ASTER DN to Radiance
    aster_band = 14
    aster_rad = aster_utils.tir_dn2rad(aster_src, aster_band)
    # set crs back
    aster_rad.rio.set_crs(aster_src.crs, inplace=True)
    # clip to geometry
    aster_rad_clipped = aster_rad.rio.clip(bounding_geometry).rename('aster')
    
    ### Compute zonal statistics and format results ###
    # Compute zonal statistics from the ASTER image, using the GOES zone labels:
    zonalstats = xrs.zonal.stats(goes_zones_repr, 
                             aster_rad_clipped, 
                             stat_funcs=['mean', 'max', 'min', 'std', 'var', 'count'])
    # Convert zonal statistics dataframe to xarray dataset
    zonalstats = xr.Dataset(zonalstats)
    
    ### Map the results from xrs.zonal.stats() back into the original zones grid ###
    # Map each zonal stat back into the original zones grid
    zonal_means = mapZonalStats(goes_zones_repr, zonalstats, 'mean')
    zonal_max = mapZonalStats(goes_zones_repr, zonalstats, 'max')
    zonal_min = mapZonalStats(goes_zones_repr, zonalstats, 'min')
    zonal_std = mapZonalStats(goes_zones_repr, zonalstats, 'std')
    zonal_var = mapZonalStats(goes_zones_repr, zonalstats, 'var')
    zonal_count = mapZonalStats(goes_zones_repr, zonalstats, 'count')
    
    ### Compute the difference between GOES Radiance and the ASTER zonal mean radiance ###
    # Compute the difference between GOES Radiance and the ASTER zonal mean radiance
    mean_diff = goes_rad_repr.values - zonal_means.values
    # Create a data array for the mean difference values
    mean_diff_da = xr.DataArray(mean_diff, name='mean_diff', dims=["y", "x"])
    
    ### Merge all zonal stats back with the original ASTER data to create a single dataset ###
    # Merge all the zonal statistics data arrays and the mean difference data array
    ds = xr.merge([aster_rad_clipped, goes_rad_repr, goes_zones_repr, zonal_means, zonal_max, zonal_min, zonal_std, zonal_var, zonal_count, mean_diff_da])
    # Clip this dataset to the ASTER image extent
    ds = ds.where(~np.isnan(aster_rad_clipped))
    # Remove data where we have overlap between a GOES pixel footprint "zone" and the edge of the ASTER image
    if zonal_count_threshold != None:
        # Use the "zonal_count" (the number of ASTER pixels in each GOES pixel footprint) to determine this
        # Example: zonal_count_threshold = 800 90m ASTER pixels is 800x90x90 square meters, or approximately 6.48 square kilometers or,
        # about 2.5x2.5 km, about the size of a full GOES-16 ABI pixel here
        # NOTE: The number of ASTER pixels per GOES pixel footprint is not constant, so there are some "edge pixels" still included here
        ds = ds.where(ds.zonal_count > zonal_count_threshold)
    
    # If an output_filepath was specified, save the resulting dataset to a netcdf file
    if output_filepath != None:
        ds.to_netcdf(output_filepath)
    
    return ds

In [ ]:
ds = upscale_aster_goes_rad_zonal_stats(aster_rad_filepath, 
                                        goes_rad_filepath, 
                                        goes_zones_filepath, 
                                        bounding_geometry=geometries, 
                                        zonal_count_threshold=800,  # 800 pixels
                                        output_filepath=None)

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols=4, figsize=(15,9), tight_layout=True)
ax = ax.flatten()

for i, data_var in enumerate(ds):
    ds[str(data_var)].plot(ax=ax[i])
    ax[i].set_title(str(data_var))